In [17]:
import time
import boto3
import sagemaker
import json
import numpy as np
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.model import Model
from sagemaker.predictor import RealTimePredictor

# Configuring the built-in Image Classification algorithm

## Configure Hyperparameters

# Number of output classes
num_classes = 2

# Number of training samples in the training set
num_training_samples = 194266

# Number of layers for the underlying neural network
num_layers = 18

# Batch size for training
mini_batch_size =  128

# Input image shape for the training data
image_shape = '3,50,50'

# Augmentation type
augmentation_type = 'crop_color_transform'

# Number of epochs
epochs = 5

# Learning rate
learning_rate = 0.01

# Enable transfer learning
use_pretrained_model = 1

## Create a unique job name 

job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

bucket = 'nitinproject1'
input_prefix = 'breast-cancer-detection/input/recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role and the training image URI for Image Classification

role = get_execution_role()
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

## Configure train timeout

train_timeout = 360000

## Creating an Estimator

sagemaker_session = sagemaker.Session()
estimator = sagemaker.estimator.Estimator(training_image, 
                                          role, 
                                          train_instance_count=instance_count,
                                          train_instance_type=instance_type,
                                          train_volume_size=volume_size_gb,
                                          train_max_run=train_timeout,
                                          output_path=output_path, 
                                          sagemaker_session=sagemaker_session,
                                          input_mode='Pipe')

estimator.set_hyperparameters(num_classes=num_classes,
                              num_training_samples=num_training_samples,
                              num_layers=num_layers,
                              mini_batch_size=mini_batch_size,
                              image_shape=image_shape,
                              augmentation_type=augmentation_type,
                              epochs=epochs,
                              learning_rate=learning_rate,
                              use_pretrained_model=use_pretrained_model)

In [18]:
s3_input_train = sagemaker.s3_input(s3_data=input_train, content_type='application/x-recordio')
s3_input_validation = sagemaker.s3_input(s3_data=input_test, content_type='application/x-recordio')

In [ ]:
# Creating a tuning job

## Defining tuning configuration

hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.001, 1.0),
    'mini_batch_size': IntegerParameter(64, 128),
    'optimizer': CategoricalParameter(['sgd', 'adam'])
}

objective_metric_name = 'validation:accuracy'
objective_type='Maximize'
max_jobs=2
max_parallel_jobs=2

## Create a unique job name

job_name_prefix = 'bcd-tuning'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Creating a hyperparameter tuner

tuner = HyperparameterTuner(estimator=estimator, 
                            objective_metric_name=objective_metric_name, 
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type, 
                            max_jobs=max_jobs, 
                            max_parallel_jobs=max_parallel_jobs)

## Launch the tuning job

tuner.fit({
    'train': s3_input_train,
    'validation': s3_input_validation
}, job_name=job_name)
tuner.wait()

.........................................................................................................................................................................................................................................................................